# **ty<span style='color:#00c49a'>ph</span>e**
## **Map letters to fingers**

This takes keyboard input data and filters letters to fingers they are pressed by. The final output are strings of letters tied to a finger that will be pasted to [this keyboard heat map generator](https://oha-ohashi.github.io/2022_1/a_ortho/ortho_heat.html) to generate heat maps each fingers.

### Imports

In [1]:
import os
import json
import re
import math

### Getting all the inputs

Change these variables according to the desired input path and the regex for the needed keys.

In [2]:
input_folder_path = "input/salvo" # folder of .json files
needed_keys = r"^[a-zA-Z]$" # only letters

Load the input .json files, open each file, filter unnecessary object keys and keyboard key presses, and append to `all_inputs`.

In [3]:
all_inputs = []

input_file_list = os.listdir(input_folder_path)
json_files = [input_file for input_file in input_file_list if input_file.endswith('.json')]

for input_file in json_files:
    with open(os.path.join(input_folder_path, input_file), 'r') as json_file:
        json_data = json.load(json_file)

        for input in range(len(json_data)):
            finger = json_data[input]['hand'].lower() + "_" + json_data[input]['finger'].lower()
            key_pressed = json_data[input]['keyPressed']

            if re.match(needed_keys, key_pressed):
                if finger == "left_thumb":
                    finger = "left_index"
                elif finger == "right_thumb":
                    finger = "right_index"

                all_inputs.append({'finger': finger, 'keyPressed': key_pressed})

### Logging the inputs for each finger

Create an object (`fingers`) with the keys being a finger with a string value of logs of their presses.

In [4]:
fingers = {}

for input in all_inputs:
    if input['finger'] not in fingers:
        fingers[input['finger']] = ""
    else:
        fingers[input['finger']] += input['keyPressed']

### Adjusting the letter proportions

Since the heat mapper does not accept very long texts, the logs need to be proportioned. If the `input['finger']` value is "AAAADD", then the proportioned string should be "AAD".

In [5]:
for finger in fingers:
    # get count of each letter
    letter_count = {}

    for letter in fingers[finger]:
        if letter not in letter_count:
            letter_count[letter] = 1
        else:
            letter_count[letter] += 1

    letter_with_lowest_count = min(letter_count, key = lambda k: letter_count[k])

    # divide each letter count by the lowest letter count and rounded up
    for letter in letter_count:
        letter_count[letter] = math.ceil(letter_count[letter] / letter_count[letter_with_lowest_count])
        
    # replace the logs with the shorter but similarly proportioned log
    new_log = ""

    for letter in letter_count:
        for count in range(letter_count[letter]):
            new_log += letter

    fingers[finger] = (new_log)

### Exporting the logs to text files

Each output .txt file contains a single string that is used to generate heat maps.

In [6]:
for finger in fingers:
    file = open('output/' + input_folder_path.split("/")[-1] + "_" + finger + ".txt", 'w')
    file.write(fingers[finger])
    file.close()

### Generating the heat maps

Use the text files in the output folder to generate the heat maps [here](https://oha-ohashi.github.io/2022_1/a_ortho/ortho_heat.html).